### Get all documents

In [ ]:
from os import listdir
from os.path import isfile, join
mypath=r'documents/Smth'
only_documents=[f for f in listdir(mypath) if isfile(join(mypath,f))]

In [ ]:
sentences = []
good = 0
bad = 0
for doc in only_documents:
    docxZip=zipfile.ZipFile(r'documents/Smth/'+doc)
    textsXML=docxZip.read('word/document.xml')
    xml_str=textsXML.decode('utf-8')
    s = xml_str
    proccess=1
    needed_text_for_docx=[]
    
    while proccess:
    sub_str=find_between(s, 'commentRangeStart', 'commentRangeEnd')
    sub_str_final = find_between(sub_str, '<w:t>', '</w:t>')
    if sub_str_final=='!':
        sub_str_final=find_between(sub_str, '<w:t xml:space="preserve">', '</w:t>')
    if sub_str == '!':
        break
    s=s.replace(sub_str,'',1)
    s=s.replace('commentRangeStartcommentRangeEnd', '', 1)
    sub_str_final=sub_str_final.lstrip().rstrip()
    needed_text_for_docx.append(sub_str_final)
    
    docxZip=zipfile.ZipFile(r'documents/Smth'+doc)
    try:
        textsXML=docxZip.read('word/comments.xml')
    except KeyError:
        continue
    xml_str=textsXML.decode('utf-8')
    
    classes_for_docx=[]
    s=xml_str
    sub_str_all=''
    proccess=1
    while proccess:
        sub_str_all=''
        sub_str=find_between(s, '<w:comment w:', '/w:comment>')
        if sub_str=='!':
            break
        s=s.replace(sub_str,'',1)
        s=s.replace('<w:comment w:/w:comment>', '', 1)
        while proccess:
            sub_str_one=find_between(sub_str, '<w:t>', '</w:t>')
            if sub_str_one == '!':
                break
            sub_str=sub_str.replace('<w:t>'+sub_str_one+'</w:t>', '', 1)
            sub_str=sub_str.lstrip().rstrip()
            sub_str_all+=sub_str_one
        classes_for_docx.append(sub_str_all)
        
    if len(classes_for_docx) != len(needed_text_for_docx):
        bad += 1
        continue
    else:
        good += 1
    text = getText(r'documents/Smth'+doc)
    #text = text.replace('\xad', '')
    #text = text.replace('данн ые', 'данные')
    #text = text.replace('\xa0№\xa0', ' № ')
    
    split_text = split_into_sentences_rus(text)
    counts_of_comments = 0
    for text in split_text:
        checker='-' #Find entities
        sentence=[] # for each sentence
        tokens=[] # for each sentence
        tags=[] # for each sentence
        for index in range(counter_of_elements, len(needed_text_for_docx)):
            checker='-'
            if needed_text_for_docx[index] in text:
                checker='+'
                text_before=text[:text.find(needed_text_for_docx[index])].split()
                if len(text_before) > 1:
                    for i in range(len(text_before)):
                        tokns.append(text_before[i])
                        tags.append('O')
                else:
                    try:
                        tokens.append(text_before[0])
                        tags.append('O')
                    except IndexError:
                        pass
                text = text[text.find(needed_text_for_docx[index]):]

                comment = needed_text_for_docx[index].split()
                if len(comment)==1:
                    tokens.append(comment[i])
                    tags.append('B-'+classes_for_docx[index])
                else:
                    for i in range(len(comment)):
                        tokens.append(comment[i])
                        if i == 0:
                            tags.append('B-'+classes_for_docx[index])
                        else:
                            tags.append('I-'+classes_for_docx[index])
                text = text.replace(needed_text_for_docx[index], '',1)
            else:
                if index==counter_of_comments:
                    checker='-'
                    break
                else:
                    checker='+'
                    text=text.split()
                    if len(text) > 1:
                        for i in range(len(text)):
                            tokens.append(text[i])
                            tags.append('O')
                    else:
                        try:
                            tokens.append(text[0])
                            tags.append('O')
                        except IndexError:
                            pass
                    break
        counter_of_comments = index
        if checker == '-':
            text=text.split()
            if len(text) > 1:
                for i in range(len(text)):
                    tokens.append(text[i])
                    tags.append('O')
            else:
                tokens.append(text[0])
                tags.append('O')
        sentence.append((tokens,tags))
        sentences.append((tokens,tags))

In [ ]:
for i in range(len(sentences)):
    sentences[i][0].append('.')
    sentences[i][1].append('O')